In [11]:
!cat /proc/meminfo

MemTotal:       32949048 kB
MemFree:        18675944 kB
MemAvailable:   20150688 kB
Buffers:          682664 kB
Cached:           997672 kB
SwapCached:            0 kB
Active:         13292612 kB
Inactive:         590480 kB
Active(anon):   12202764 kB
Inactive(anon):       48 kB
Active(file):    1089848 kB
Inactive(file):   590432 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                84 kB
Writeback:             0 kB
AnonPages:      12202820 kB
Mapped:           155392 kB
Shmem:                60 kB
Slab:             282336 kB
SReclaimable:     258372 kB
SUnreclaim:        23964 kB
KernelStack:        4464 kB
PageTables:        30088 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    16474524 kB
Committed_AS:   11514496 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

In [33]:
!ps -fea | grep python

ec2-user  5277  5252  0 Sep04 ?        00:01:53 /home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python /home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/jupyter-notebook --notebook-dir=/home/ec2-user/SageMaker/ --ip=0.0.0.0 --NotebookApp.token= --port=8443 --NotebookApp.disable_check_xsrf=True
ec2-user 14394  5277  0 Sep04 ?        00:00:01 /home/ec2-user/anaconda3/envs/python3/bin/python -m ipykernel -f /home/ec2-user/.local/share/jupyter/runtime/kernel-0697d8d3-db51-4d35-bd8f-010af26137c4.json
ec2-user 22925  5277 37 Sep04 ?        01:44:46 /home/ec2-user/anaconda3/envs/python3/bin/python -m ipykernel -f /home/ec2-user/.local/share/jupyter/runtime/kernel-5e222582-70ec-4279-a521-72379c4d52ea.json
ec2-user 23239  5277  0 Sep04 ?        00:00:00 /home/ec2-user/anaconda3/envs/python3/bin/python -m ipykernel -f /home/ec2-user/.local/share/jupyter/runtime/kernel-5c03c7f8-5922-4b99-a26c-d5f9383734c8.json
ec2-user 23268  5277  0 Sep04 ?        00:00:00 /home/ec2-user/anaconda3/envs/pytho

In [2]:
!pip install xgboost

    100% |████████████████████████████████| 15.8MB 3.2MB/s eta 0:00:01
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import gc
import pickle

from sklearn import preprocessing as prp
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import (accuracy_score, r2_score, confusion_matrix,
f1_score, recall_score, precision_score)

import os
import boto3
import re

import xgboost as xgb

# ruta de trabajo en s3
bucket = 'boi-banregio'
target = 'BMI'
strategy = 'median'
feat_part_number = 40

# s3 resource
s3_bucket_resource = boto3.resource('s3').Bucket(bucket)

In [4]:
# Aquí colocamos la ruta de nuestro directorio, el nombre de nuestro archivo y si tiene o no cabecera
prefix = 'datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION'

## **Variables extras**

In [5]:
extra_features_file_name = 'EXTRA_FEATURES/JAT_MCV_VAR_VARIABLES_DIASATRASO.csv'
extra_features_file_key = "{}/{}".format(prefix, extra_features_file_name)
extra_features_obj = s3_bucket_resource.Object(extra_features_file_key).get()
extra_features_data = pd.read_csv(io.BytesIO(extra_features_obj['Body'].read()))

In [6]:
display(extra_features_data.shape)
display(extra_features_data.columns)
display(extra_features_data.head())
print("Hay {} nulos.".format(extra_features_data.isnull().sum().sum()))

(425287, 4)

Index(['DIAS_ATRASO', 'DPCV_CAP', 'DPCV_INT', 'DPCV_MIN'], dtype='object')

,DIAS_ATRASO,DPCV_CAP,DPCV_INT,DPCV_MIN
0,0,61,61,61
1,0,61,61,61
2,0,61,61,61
3,0,61,61,61
4,0,61,61,61


Hay 0 nulos.


## **Imputación no necesria**

In [10]:
imputer = prp.Imputer(missing_values='NaN', strategy=strategy, axis=0)
scaler = prp.RobustScaler()

In [11]:
imputer = imputer.fit(extra_features_data)
extra_features_data.loc[:,:] = imputer.transform(extra_features_data)
extra_features_data.loc[:,:] = scaler.fit_transform(extra_features_data)


NameError: name 'data_part' is not defined

In [17]:
display(extra_features_data.describe())

,DIAS_ATRASO,DPCV_CAP,DPCV_INT,DPCV_MIN
count,425287.000000,425287.000000,425287.000000,425287.000000
mean,1.938785,55.445659,73.343079,55.445659
std,11.199988,24.343744,19.728195,24.343744
min,0.000000,-537.000000,-537.000000,-537.000000
25%,0.000000,30.000000,61.000000,30.000000
50%,0.000000,61.000000,82.000000,61.000000
75%,0.000000,61.000000,90.000000,61.000000
max,598.000000,90.000000,90.000000,90.000000


## **No es necesario guardar un archivo de preprocesamiento**

In [15]:
 print('Uploading processed data')
data_processed_part_file_name = "EXTRA_FEATURES/PROCESSED_DATA/VARIABLES_DIASATRASO_PREPROCESSED.csv"
data_processed_part_obj_key =  "{}/{}".format(prefix, data_processed_part_file_name)    

f_str = io.StringIO()
extra_features_data.to_csv(f_str, index=False)
s3_bucket_resource.Object(data_processed_part_obj_key).put(Body=f_str.getvalue())

Uploading processed data


{'ResponseMetadata': {'RequestId': '15879FB564C41483',
  'HostId': 'J0t8bj5/SMvLZ9tcl9PenyfVget6uXKvpgjuo5tdOnHsED932G7hyOq8pBezLppI',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'J0t8bj5/SMvLZ9tcl9PenyfVget6uXKvpgjuo5tdOnHsED932G7hyOq8pBezLppI',
   'x-amz-request-id': '15879FB564C41483',
   'date': 'Tue, 04 Sep 2018 23:24:26 GMT',
   'etag': '"c7b06f7e6e0789ab860a03c958108629"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"c7b06f7e6e0789ab860a03c958108629"'}

## **Concatenamos**

In [7]:
model_features_file_name = "MODEL_DATASET/MODEL_FEATURES.csv"
model_features_file_key = "{}/{}".format(prefix, model_features_file_name)
model_features_obj = s3_bucket_resource.Object(model_features_file_key).get()
model_features_data = pd.read_csv(io.BytesIO(model_features_obj['Body'].read()))

In [8]:
features_data =pd.concat([model_features_data, extra_features_data], axis=1) 

In [9]:
features_data.columns

Index(['AVG_MAX_DIAS_PARA_PAGO_U06M', 'AVG2_VP_MES_2',
       'AVG_R_BAL_BY_CRED_LIM_12M', 'DEUDA_TOT', 'MIN_R_BAL_BY_MAX_BAL_03M',
       'MAX_VENC1M_BRG_OP_ACC_06M', 'AVG_AVG2_VP_MES_U03M_2',
       'AVG_VENC1M_BK_OP_ACC', 'MIN_AVG2_VP_PER_U03M_2', 'NUM_VIGENTEH_12M',
       ...
       'MAX_PCT_VENC1M_BRG_CS_OP_ACC_06M', 'MIN_AVG_HI_CALCAR_CEIL_12M',
       'MAX_VIGENTEC_BK_CS_OP_ACC_03M', 'AVG_MAX_VP_MES_U06M',
       'AVG_PCT_VENC1M_BK_R_OP_ACC_12M', 'MAX_SALINI_LSE_CL_ACC',
       'DIAS_ATRASO', 'DPCV_CAP', 'DPCV_INT', 'DPCV_MIN'],
      dtype='object', length=540)

In [10]:
 print('Uploading processed data')
data_processed_part_file_name = "MODEL_DATASET/MODEL_FEATURES+EXTRAS.csv"
data_processed_part_obj_key =  "{}/{}".format(prefix, data_processed_part_file_name)    

f_str = io.StringIO()
features_data.to_csv(f_str, index=False)
s3_bucket_resource.Object(data_processed_part_obj_key).put(Body=f_str.getvalue())

Uploading processed data


{'ResponseMetadata': {'RequestId': 'AC552D947AF16969',
  'HostId': '7y/jOOG9rGE1U98YxEfOQHfg3HELNZbYv7xlkEsTTXB7H3H3X+rAk2tBq7Pyj0v/ZPHwlXG4rXM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '7y/jOOG9rGE1U98YxEfOQHfg3HELNZbYv7xlkEsTTXB7H3H3X+rAk2tBq7Pyj0v/ZPHwlXG4rXM=',
   'x-amz-request-id': 'AC552D947AF16969',
   'date': 'Fri, 21 Sep 2018 23:18:50 GMT',
   'etag': '"0c4799f8a1efa863533086fc56307931"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"0c4799f8a1efa863533086fc56307931"'}